In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
!pip install gdown

  Using cached gdown-4.7.1-py3-none-any.whl (15 kB)


In [15]:
import json
from matplotlib import pyplot as plt
import glob
import os
import paddleocr
from paddleocr import draw_ocr
from PIL import Image
import numpy as np
import math
import sys
import gdown
import tarfile

sys.path.append('../../ocr_ensemble/')
from label_studio_utils import convert_labelstudio2json

In [4]:
with open('../../data/label_studio/done.json', 'r') as f:
    labelled = json.load(f)

In [5]:
json_dataset = convert_labelstudio2json(labelled)

In [6]:
len(json_dataset)

405

In [7]:
json_dataset[0]

{'ocr_annotation': {'bounding_boxes': [[[160.62745098039215,
     725.3333333333335],
    [860.8627450980393, 725.3333333333335],
    [860.8627450980393, 800.6274509803923],
    [160.62745098039215, 800.6274509803923]],
   [[364.7379245557456, 802.6046279679395],
    [655.9024843694806, 802.6046279679395],
    [655.9024843694806, 835.0806750240869],
    [364.7379245557456, 835.0806750240869]],
   [[170.93044822256567, 728.0370942812983],
    [851.4868624420402, 728.0370942812983],
    [851.4868624420402, 794.5100463678516],
    [170.93044822256567, 794.5100463678516]],
   [[370.34930448222565, 804.0061823802164],
    [661.564142194745, 804.0061823802164],
    [661.564142194745, 838.8253477588871],
    [370.34930448222565, 838.8253477588871]]],
  'text': ['GUDNEX ENTERPRISES',
   'POSTED ON JIJI',
   'GUDNEX ENTERPRISES',
   'POSTED ON JIJI'],
  'rotation': [0, 0, 0, 0]},
 'width': 1024,
 'height': 1024,
 'label_studio_fname': '/data/upload/1/60569e35-013370001.jpg',
 'label_studio_id':

# check whether we can render the bounding boxes

In [17]:
ls_folder = '../../data/label_studio/'
image_folder = ls_folder + '01337/'

In [16]:
file_id = '15VMiPXZqIqUGJjM-lUoEPAvANnF9qEhu'
url = f'https://drive.google.com/uc?id={file_id}'
output = ls_folder+'01337.tar'
tar_file_path = gdown.download(url, output, quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=15VMiPXZqIqUGJjM-lUoEPAvANnF9qEhu
From (redirected): https://drive.google.com/uc?id=15VMiPXZqIqUGJjM-lUoEPAvANnF9qEhu&confirm=t&uuid=1bf95494-d611-41cb-8c2d-10ff639ba774
To: C:\Users\Chris Wendler\Documents\git\2023\OCR-ensemble\data\label_studio\01337.tar
100%|██████████████████████████████████████████████████████████████████████████████| 1.16G/1.16G [00:10<00:00, 110MB/s]


In [18]:
with tarfile.open(tar_file_path, 'r') as tar_ref:
    tar_ref.extractall(image_folder)

In [19]:
def my_draw_ocr(imname, annotation,font_path='../../fonts/simfang.ttf'):
    bboxes = annotation['bounding_boxes']
    txts = annotation['text']
    scores = annotation['confidences'] if 'confidences' in annotation else [1]*len(txts)
    im_show = draw_ocr(Image.open(imname), bboxes, txts, scores, font_path=font_path)
    im_show = Image.fromarray(im_show)
    im_show.show()

In [20]:
for d in json_dataset:
    imname = image_folder + d['label_studio_fname'].split('-')[1]
    if sum(d['ocr_annotation']['rotation']) != 0:
        my_draw_ocr(imname, d['ocr_annotation'])

KeyboardInterrupt: 